<a href="https://colab.research.google.com/github/jennif8r/LllForIFPR/blob/main/testDois.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Passo 1: Extração de informação de um site

In [1]:
import requests
from bs4 import BeautifulSoup

# URL do site que queremos extrair informações
url = 'https://ifpr.edu.br/colombo/'

# Enviando uma solicitação GET para o site
response = requests.get(url)
html = response.text

# Usando BeautifulSoup para analisar o HTML
soup = BeautifulSoup(html, 'html.parser')

# Extraindo o texto do site
text = soup.get_text()

print(text)




















Campus Colombo – IFPR – Educação Pública, Gratuita e de Qualidade



















 







Portal do Governo Brasileiro


















Instituto Federal do Paraná
Marca do IFPR











INSTITUTO FEDERAL

Paraná
Campus Colombo













Pesquisar em ifpr.edu.br/colombo					Pesquisar ifpr.edu.br/colombo:



Pesquisar

<Pesquisa Pública




Início
Estude no IFPR
Nossos Cursos

Técnicos Integrados
Técnicos Subsequentes
Superiores
Pronatec
Educação a Distância


 











InstitucionalO Instituto

O Instituto
O Campus Colombo
Fale Conosco


Nossos Cursos

Técnico Integrado em Administração
Técnico Integrado em Alimentos
Técnico Integrado em Informática
Técnico Subsequente em Administração
Tecnologia em Alimentos
Tecnologia em Análise e Desenvolvimento de Sistemas
Tecnologia em Processos Gerenciais


Pesquisa e Extensão

Comitê de Pesquisa e Extensão


Órgãos Colegiados

CODIC – Colégio de Dirigentes do Campus
NAPNE – Núcleo de Apoio a Pessoas com Necessidade

#Passo 2: Pré-processamento do texto extraído

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# Baixando recursos necessários do nltk
nltk.download('punkt')
nltk.download('stopwords')

# Função de pré-processamento
def preprocess_text(text):
    # Tokenização
    tokens = word_tokenize(text)

    # Remoção de pontuação e stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalnum() and word.lower() not in stop_words]

    return tokens

# Pré-processando o texto extraído
processed_text = preprocess_text(text)
print(processed_text)


['Campus', 'Colombo', 'IFPR', 'Educação', 'Pública', 'Gratuita', 'e', 'de', 'Qualidade', 'Portal', 'Governo', 'Brasileiro', 'Instituto', 'Federal', 'Paraná', 'Marca', 'IFPR', 'INSTITUTO', 'FEDERAL', 'Paraná', 'Campus', 'Colombo', 'Pesquisar', 'em', 'Pesquisar', 'Pesquisar', 'Pesquisa', 'Pública', 'Início', 'Estude', 'IFPR', 'Nossos', 'Cursos', 'Técnicos', 'Integrados', 'Técnicos', 'Subsequentes', 'Superiores', 'Pronatec', 'Educação', 'Distância', 'InstitucionalO', 'Instituto', 'Instituto', 'Campus', 'Colombo', 'Fale', 'Conosco', 'Nossos', 'Cursos', 'Técnico', 'Integrado', 'em', 'Administração', 'Técnico', 'Integrado', 'em', 'Alimentos', 'Técnico', 'Integrado', 'em', 'Informática', 'Técnico', 'Subsequente', 'em', 'Administração', 'Tecnologia', 'em', 'Alimentos', 'Tecnologia', 'em', 'Análise', 'e', 'Desenvolvimento', 'de', 'Sistemas', 'Tecnologia', 'em', 'Processos', 'Gerenciais', 'Pesquisa', 'e', 'Extensão', 'Comitê', 'de', 'Pesquisa', 'e', 'Extensão', 'Órgãos', 'Colegiados', 'CODIC', '

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Passo 3: Treinamento de um modelo LLM
Aqui, vamos usar um modelo pré-treinado da Hugging Face e ajustar ele com os dados extraídos. Para simplificação, usaremos o modelo GPT-2.

In [3]:
!pip install dataset

In [4]:
!pip install transformers datasets

In [19]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

In [20]:
# Convertendo o texto pré-processado de volta para uma string
training_text = ' '.join(processed_text)

In [22]:
# Criando um dataset com o texto de treinamento
dataset = Dataset.from_dict({'text': [training_text]})

In [23]:
# Inicializando o tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [24]:
# Adicionando o token de padding
tokenizer.pad_token = tokenizer.eos_token

In [26]:
# Tokenizando o dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['text'])


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [28]:
# Função para contar tokens
def count_tokens(examples):
    examples['num_tokens'] = [len(input_id) for input_id in examples['input_ids']]
    return examples

# Contando tokens no dataset tokenizado
tokenized_datasets = tokenized_datasets.map(count_tokens, batched=True)

# Exibindo a contagem de tokens
print(tokenized_datasets['num_tokens'])

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

[1024]


In [30]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets,
)

In [31]:
# Treinamento
trainer.train()

Step,Training Loss


TrainOutput(global_step=1, training_loss=4.456427574157715, metrics={'train_runtime': 48.7074, 'train_samples_per_second': 0.021, 'train_steps_per_second': 0.021, 'total_flos': 522584064000.0, 'train_loss': 4.456427574157715, 'epoch': 1.0})

In [34]:
# Salvando o modelo treinado e o tokenizer
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.json',
 './results/merges.txt',
 './results/added_tokens.json')

#Testando o Modelo Treinado

In [35]:
from transformers import pipeline

In [37]:
# Carregando o modelo treinado e o tokenizer
model_path = "./results"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

In [38]:
# Criando o pipeline
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [40]:
# Fazendo perguntas ao modelo
question = "cursos"
response = qa_pipeline(question)

print(response[0]['generated_text'])

cursos," according to a tweet from his representative. He added, "I'd like to thank them for their hard work in the community and for doing our part to protect us from terrorism. Also, I would like to express our gratitude to
